In [ ]:
client = bigquery.Client(project='zeta-medley-405005')
table_id =  "zeta-medley-405005.dep_raw.order_items"






def load_departments():
    print(f" INICIO LOAD CATEGORIES")
    
    dbconnect = get_connect_mongo()
    dbname=dbconnect["retail_db"]
    collection_name = dbname["departments"] 
    departments = collection_name.find({})
    departments_df = DataFrame(departments)
    dbconnect.close()
    
    departments_df = departments_df.drop(columns=['_id'])
    
    departments_rows=len(departments_df)
    print(f" Se obtuvo  {departments_rows}  Filas")

    departments_rows=len(departments_df)
    
    if departments_rows>0 :
        client = bigquery.Client(project='zeta-medley-405005')
        table_id =  "zeta-medley-405005.dep_raw.departments"
        
        job_config = bigquery.LoadJobConfig(
            schema=[
                bigquery.SchemaField("department_id", bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField("department_name", bigquery.enums.SqlTypeNames.STRING)
            ],
            write_disposition="WRITE_TRUNCATE",
        )
    
    
        job = client.load_table_from_dataframe(
            departments_df, table_id, job_config=job_config
        )  
        job.result()  # Wait for the job to complete.
    
        table = client.get_table(table_id)  # Make an API request.
        print(
            "Loaded {} rows and {} columns to {}".format(
                table.num_rows, len(table.schema), table_id
            )
        )
    else : 
        print('alerta no hay registros en la tabla departments')